In [1]:
import pandas as pd
import numpy as np
from plotly.graph_objs import *
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, EffectScatter

In [2]:
df = pd.read_csv("BTC_daily.csv")               # 可改为ETH_daily.csv
df["Close_1_shift"] = df["Close"].shift(1) 
K = 2                                           # K倍标准差
N = 20                                          # N日移动平均线

In [3]:
df["sma"] = np.round(df["Close"].rolling(N).mean(),4)
df["stdev"] = np.round(df["Close"].rolling(N).std(),4) 
df["upband"]  = np.round(df["sma"] + K*df["stdev"],4)
df["downband"] = np.round(df["sma"] - K*df["stdev"],4)

In [4]:
money = [1000.0 for i in range(20)]  # initial money value in USDT
crypto_value = [0 for i in range(20)]
crypto = 0 
fees = 0.001           # fees as 0.1%
positions = []  
success_history = []  
failure_history = []  


In [5]:
#布林通道顺势策略

for i in range(21, df.shape[0]):
    if crypto == 0 and df["Close"].iloc[i-1] > df["upband"].iloc[i-1] and i != df.shape[0] - 1:     # 1.突破上轨线，建仓

        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        crypto = np.round(money[-1]*(1.0 - fees)/ price, 4)
        crypto_value += [crypto*df["Close"].iloc[i]]
        money += [0]
        positions += [{ "date": df.Date.iloc[i], "price": price}]
        
    elif crypto == 0:                                                                    # 2.空仓不动
        money += [money[-1]]
        crypto_value += [0]
            
    elif crypto > 0 and (df["Close"].iloc[i-1] < df["downband"].iloc[i-1] or i == df.shape[0] - 1 ):   # 3.突破下轨线，清仓       
        
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        money += [crypto * price *(1-fees)]
        crypto = 0
        crypto_value +=[0]
        
        if positions[-1]["price"] < price:
            success_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        else:
            failure_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        positions = []

    elif crypto > 0:                                                             # 4.持仓不动
        money += [0]
        crypto_value += [crypto * df["Close"].iloc[i]]


In [6]:
success_rate = 0
if len(success_history) + len(failure_history) > 0:
    success_rate = len(success_history) / (len(failure_history) + len(success_history))


In [7]:
print("-" * 50)
print("Benchmark",(df.Close.iloc[df.shape[0]-1]/df.Close.iloc[0]-1)*1000+1000)
print("Capital at the end", np.round(money[-1], 2))
print("Accumulated return at the end", np.round(money[-1]/money[0]*100, 4),"%")
print("Win ratio", success_rate )
print("-" * 50)
print("Summary of % change in positions")
percent_change = []
positions = []
for h in [failure_history, success_history]:
    for position in h:
        percent_change.append(np.round((position["price"][1] - position["price"][0])/ position["price"][0]* 100.0, 2,))
        print("Percent change in position", percent_change[-1])
        positions += [position]
        
print("-" * 50)
print("Total trade times", len(percent_change))
print("Average return per trade", np.round(sum(percent_change)/len(percent_change),2),"%")
print("Max win", max(percent_change),"%")
print("Max loss", min(percent_change),"%")

--------------------------------------------------
Benchmark 5583.640820180951
Capital at the end 6472.19
Accumulated return at the end 647.2186 %
Win ratio 0.6666666666666666
--------------------------------------------------
Summary of % change in positions
Percent change in position -3.08
Percent change in position -9.27
Percent change in position 41.7
Percent change in position 353.69
Percent change in position 9.88
Percent change in position 5.36
--------------------------------------------------
Total trade times 6
Average return per trade 66.38 %
Max win 353.69 %
Max loss -9.27 %


In [8]:
attr=[str(t) for t in df.Date]
v1=np.array(df.loc[:,['Open','Close','Low','High']]).tolist()
v2=np.array(df.sma).tolist()
v3=np.array(df.upband).tolist()
v4=np.array(df.downband).tolist()
bd = [str(i["date"][0]) for i in positions]
sd = [str(i["date"][1]) for i in positions]
bp = [np.round((i["price"][0]),2) for i in positions]
sp = [np.round((i["price"][1]),2) for i in positions]

kline1 = (
    Kline()
    .add_xaxis(attr)
    .add_yaxis("BTC",v1)                        #标题BTC，如果使用ETH则改为ETH
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%")],
        title_opts=opts.TitleOpts(title="boll_1"),
    ) 
)

line1 = (
    Line()
    .add_xaxis(attr)
    .add_yaxis("mid_line",v2 , is_smooth = True,)
    .add_yaxis("upband",v3 , is_smooth = True,)
    .add_yaxis("downband",v4 , is_smooth = True,)
)

cb = (
    EffectScatter()
    .add_xaxis(bd)
    .add_yaxis("Buy", bp)
)

cs = (
    EffectScatter()
    .add_xaxis(sd)
    .add_yaxis("Sell", sp)
)

kline1.overlap(line1)
kline1.overlap(cb)
kline1.overlap(cs)
kline1.render("p1_boll_1.html")

'F:\\jupyter\\p1_boll_1.html'

In [9]:
rtn = [np.round((crypto_value[i]+money[i])/money[0],4) for i in range(df.shape[0]-1)]
ben = [np.round(j/df.Close.iloc[0],4) for j in df.Close]

line2 = (
    Line()
    .add_xaxis(attr)
    .add_yaxis("rtn",rtn , is_smooth = True,)
    .add_yaxis("ben",ben , is_smooth = True,) 
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%")],
        title_opts=opts.TitleOpts(title="boll_1"),
    ) 
)
line2.render("p2_boll_1.html")

'F:\\jupyter\\p2_boll_1.html'